# Schema Verification Notebook - Column Case Sensitivity Check

**Purpose**: Verify column existence and correct case for all schemas defined in `schemas.py`.

**Problem**: BronzeReader lowercases all columns, but schemas define columns with specific cases. This causes NULL columns when case doesn't match the actual CSV files.

**Solution**: 
1. Read each file WITHOUT schema  
2. Compare actual column names with schema definitions  
3. Identify case mismatches and missing columns  
4. Correct schemas to match actual CSV column names

**Order**: Follows schema definition order in `config/schemas.py`

---

## Setup

In [ ]:
from pyspark.sql import SparkSession
from azfr_fsspec_utils import fspath
import azfr_fsspec_abfs

azfr_fsspec_abfs.use()

spark = SparkSession.builder \
    .appName("Schema_Verification") \
    .config("spark.sql.adaptive.enabled", "true") \
    .getOrCreate()

print(f"✓ Spark {spark.version} initialized")

In [ ]:
# Configuration
DATALAKE_BASE = "abfss://shared@azfrdatalab.dfs.core.windows.net/ABR/P4D/ADC/DATAMARTS/CONSTRUCTION"
VISION = "202509"
YEAR = VISION[:4]
MONTH = VISION[4:6]

MONTHLY_PATH = f"{DATALAKE_BASE}/bronze/{YEAR}/{MONTH}"
REF_PATH = f"{DATALAKE_BASE}/bronze/ref"

print(f"Monthly: {MONTHLY_PATH}")
print(f"Reference: {REF_PATH}")

## Helper Functions

In [ ]:
def verify_schema(file_path, schema_name, schema_columns, description):
    """
    Verify schema columns against actual CSV file.
    
    Args:
        file_path: Path to CSV file
        schema_name: Name of schema (e.g., 'IPF_AZ_SCHEMA')
        schema_columns: List of column names from schema definition
        description: Human-readable description
    """
    print(f"\n{'='*100}")
    print(f"SCHEMA: {schema_name}")
    print(f"FILE: {description}")
    print(f"PATH: {file_path}")
    print(f"{'='*100}")
    
    try:
        # Read CSV without schema
        df = spark.read.csv(
            file_path,
            sep='|',
            header=True,
            inferSchema=False,  # Just check column names
            encoding="LATIN9"
        )
        
        actual_cols = df.columns
        record_count = df.count()
        
        print(f"\n✓ File found: {record_count:,} records, {len(actual_cols)} columns")
        
        # Build case-insensitive comparison maps
        actual_map = {c.lower(): c for c in actual_cols}
        schema_map = {c.lower(): c for c in schema_columns}
        
        # Find matches, mismatches, missing
        perfect_matches = []
        case_mismatches = []
        missing = []
        
        for schema_col in schema_columns:
            schema_lower = schema_col.lower()
            if schema_lower in actual_map:
                actual_col = actual_map[schema_lower]
                if actual_col == schema_col:
                    perfect_matches.append(schema_col)
                else:
                    case_mismatches.append((schema_col, actual_col))
            else:
                missing.append(schema_col)
        
        # Extra columns not in schema
        extra = [c for c in actual_cols if c.lower() not in schema_map]
        
        # Report
        print(f"\n{'─'*100}")
        print(f"VERIFICATION RESULTS:")
        print(f"{'─'*100}")
        
        print(f"\n✓ Perfect matches: {len(perfect_matches)}/{len(schema_columns)}")
        if len(perfect_matches) < len(schema_columns) and perfect_matches:
            print(f"  Sample: {', '.join(perfect_matches[:5])}...")
        
        if case_mismatches:
            print(f"\n⚠ CASE MISMATCHES: {len(case_mismatches)} (MUST FIX!)")
            print(f"  Schema → Actual CSV:")
            for schema_col, actual_col in case_mismatches:
                print(f"    '{schema_col}' → '{actual_col}'")
        
        if missing:
            print(f"\n✗ MISSING in CSV: {len(missing)} columns")
            for col in missing:
                print(f"    '{col}'")
        
        if extra:
            print(f"\n⚠ Extra in CSV (not in schema): {len(extra)}")
            if len(extra) > 10:
                print(f"  Sample: {', '.join(extra[:10])}... ({len(extra)-10} more)")
            else:
                print(f"  {', '.join(extra)}")
        
        # Summary
        if not case_mismatches and not missing:
            print(f"\n✅ SCHEMA CORRECT - All columns match!")
        elif case_mismatches and not missing:
            print(f"\n❌ ACTION REQUIRED: Fix {len(case_mismatches)} case mismatches in schemas.py")
        elif missing:
            print(f"\n❌ ACTION REQUIRED: {len(missing)} columns missing from CSV file")
        
        print(f"{'─'*100}")
        
        return {
            'schema': schema_name,
            'file': description,
            'perfect': len(perfect_matches),
            'case_mismatch': len(case_mismatches),
            'missing': len(missing),
            'extra': len(extra),
            'mismatches': case_mismatches
        }
        
    except Exception as e:
        print(f"\n✗ ERROR: {e}")
        print(f"{'─'*100}")
        return {'schema': schema_name, 'error': str(e)}

print("✓ Helper function defined")

---
# Schema Verification (Following schemas.py order)
---

## 1. RF_FR1_PRM_DTL_MIDCORP_M_SCHEMA - One BI Premium Data

In [ ]:
# From schemas.py lines 35-51
schema_cols = [
    "CD_NIV_2_STC", "CD_INT_STC", "NU_CNT_PRM", "CD_PRD_PRM", "CD_STATU_CTS",
    "DT_CPTA_CTS", "DT_EMIS_CTS", "DT_ANNU_CTS", "MT_HT_CTS", "MT_CMS_CTS",
    "CD_CAT_MIN", "CD_GAR_PRINC", "CD_GAR_PROSPCTIV", "NU_EX_RATT_CTS", "CD_MARCHE"
]

result1 = verify_schema(
    f"{MONTHLY_PATH}/rf_fr1_prm_dtl_midcorp_m_*.csv",
    "RF_FR1_PRM_DTL_MIDCORP_M_SCHEMA",
    schema_cols,
    "One BI Premium Detail - Emissions data"
)

## 2. IPF_AZ_SCHEMA - IMS Portfolio (PTF16/PTF36)

In [ ]:
# From schemas.py lines 59-213 (abbreviated for notebook)
schema_cols = [
    "CDPROD", "NOPOL", "NOCLT", "NMCLT", "NOINT", "NMACTA",
    "POSACTA", "RUEACTA", "CEDIACTA",
    "DTCREPOL", "DTEFFAN", "DTTRAAN", "DTRESILP", "DTTRAAR",
    "DTTYPLI1", "DTTYPLI2", "DTTYPLI3", "DTECHANN", "DTOUCHAN",
    "DTRCPPR", "DTRECTRX", "DTRCPRE", "DTEFSITT",
    "CDNATP", "TXCEDE", "PTGST", "CMARCH", "CDSITP", "CSEGT", "CSSEGT", "CDRI",
    "CDTYPLI1", "CDTYPLI2", "CDTYPLI3",
    "MTPRPRTO", "PRCDCIE", "MTCAF", "FNCMACA", "MTSMPR",
    "CDPOLQPL", "CDTPCOA", "CDCIEORI",
    "CDPOLRVI", "CDGREV",
    "CDSITMGR", "CDGECENT", "CDMOTRES", "NOPOLLI1", "CDCASRES",
    # Capital fields (14 pairs)
    "MTCAPI1", "MTCAPI2", "MTCAPI3", "MTCAPI4", "MTCAPI5", "MTCAPI6", "MTCAPI7",
    "MTCAPI8", "MTCAPI9", "MTCAPI10", "MTCAPI11", "MTCAPI12", "MTCAPI13", "MTCAPI14",
    "LBCAPI1", "LBCAPI2", "LBCAPI3", "LBCAPI4", "LBCAPI5", "LBCAPI6", "LBCAPI7",
    "LBCAPI8", "LBCAPI9", "LBCAPI10", "LBCAPI11", "LBCAPI12", "LBCAPI13", "LBCAPI14",
    # Provider fields (14 pairs)
    "CDPRVB1", "CDPRVB2", "CDPRVB3", "CDPRVB4", "CDPRVB5", "CDPRVB6", "CDPRVB7",
    "CDPRVB8", "CDPRVB9", "CDPRVB10", "CDPRVB11", "CDPRVB12", "CDPRVB13", "CDPRVB14",
    "PRPRVC1", "PRPRVC2", "PRPRVC3", "PRPRVC4", "PRPRVC5", "PRPRVC6", "PRPRVC7",
    "PRPRVC8", "PRPRVC9", "PRPRVC10", "PRPRVC11", "PRPRVC12", "PRPRVC13", "PRPRVC14",
    # Risk fields
    "CDFRACT", "QUARISQ", "NMRISQ", "NMSRISQ", "RESRISQ", "RUERISQ", "LIDIRISQ", "POSRISQ", "VILRISQ", "CDREG",
    # NAF / Activity
    "CDNAF", "CDTRE", "CDACTPRO", "ACTPRIN",
    # IRD/Transaction
    "tydris1", "OPAPOFFR", "CTDEFTRA", "CTPRVTRV", "LBNATTRV", "DSTCSC", "LBQLTSOU"
]

result2a = verify_schema(
    f"{MONTHLY_PATH}/ipf16.csv",
    "IPF_AZ_SCHEMA",
    schema_cols,
    "IPF16 - Agent Portfolio"
)

In [ ]:
result2b = verify_schema(
    f"{MONTHLY_PATH}/ipf36.csv",
    "IPF_AZ_SCHEMA",
    schema_cols,
    "IPF36 - Courtage Portfolio"
)

## 3. IPFM99_AZ_SCHEMA

In [ ]:
# From schemas.py lines 221-231
schema_cols = [
    "CDPROD", "NOPOL", "NOINT", "CDACPR1", "CDACPR2",
    "MTCA", "MTCAENP", "MTCASST", "MTCAVNT"
]

result3 = verify_schema(
    f"{MONTHLY_PATH}/*SPEIPFM99_IPF_*.csv.gz",
    "IPFM99_AZ_SCHEMA",
    schema_cols,
    "IPFM99 - Product 01099 CA amounts"
)

## 4. IRD_RISK_COMMON_SCHEMA (Q45/Q46)

In [ ]:
# From schemas.py lines 239-248
schema_cols = [
    "NOPOL", "DTOUCHAN", "DTRECTRX", "DTREFFIN",
    "CTPRVTRV", "CTDEFTRA", "LBNATTRV", "LBDSTCSC"
]

result4a = verify_schema(
    f"{MONTHLY_PATH}/ird_risk_q45_*.csv",
    "IRD_RISK_COMMON_SCHEMA",
    schema_cols,
    "IRD Risk Q45"
)

result4b = verify_schema(
    f"{MONTHLY_PATH}/ird_risk_q46_*.csv",
    "IRD_RISK_COMMON_SCHEMA",
    schema_cols,
    "IRD Risk Q46"
)

## 5. IRD_RISK_QAN_SCHEMA

In [ ]:
# From schemas.py lines 251-257
schema_cols = [
    "NOPOL", "DTOUCHAN", "CTPRVTRV", "CTDEFTRA", "LBNATTRV"
]

result5 = verify_schema(
    f"{MONTHLY_PATH}/ird_risk_qan_*.csv",
    "IRD_RISK_QAN_SCHEMA",
    schema_cols,
    "IRD Risk QAN"
)

## 6. INCENDCU_SCHEMA - AZEC Fire Insurance

In [ ]:
# From schemas.py lines 264-271
schema_cols = [
    "POLICE", "PRODUIT", "COD_NAF", "COD_TRE",
    "MT_BASPE", "MT_BASDI"
]

result6 = verify_schema(
    f"{REF_PATH}/incendcu.csv",
    "INCENDCU_SCHEMA",
    schema_cols,
    "INCENDCU - Fire insurance / NAF / PE-RD"
)

## 7. CONSTRCU_SCHEMA - Construction Site Data

In [ ]:
# From schemas.py lines 278-292
schema_cols = [
    "DATFINCH", "DATOUVCH", "DATRECEP", "DEST_LOC", "FORMULE",
    "LDESTLOC", "LQUALITE", "LTYPMAR1", "MNT_GLOB", "NAT_CNT",
    "POLICE", "PRODUIT", "TYPMARC1"
]

result7 = verify_schema(
    f"{REF_PATH}/constrcu.csv",
    "CONSTRCU_SCHEMA",
    schema_cols,
    "CONSTRCU - Construction site details"
)

## 8. SEGMPRDT_SCHEMA - Product Segmentation

In [ ]:
# From schemas.py lines 314-323
schema_cols = [
    "CMARCH", "CPROD", "CSEG", "CSSSEG",
    "LMARCH", "LPROD", "LSEG", "LSSSEG"
]

result8a = verify_schema(
    f"{REF_PATH}/prdpfa1.csv",
    "SEGMPRDT_SCHEMA",
    schema_cols,
    "PRDPFA1 - Product segmentation (Agent)"
)

result8b = verify_schema(
    f"{REF_PATH}/prdpfa3.csv",
    "SEGMPRDT_SCHEMA",
    schema_cols,
    "PRDPFA3 - Product segmentation (Courtage)"
)

## 9. LOB_SCHEMA - Line of Business

In [ ]:
# From schemas.py lines 330-345
schema_cols = [
    "PRODUIT", "CDPROD", "CPROD", "CMARCH", "LMARCH", "LMARCH2",
    "CSEG", "LSEG", "LSEG2", "CSSSEG", "LSSSEG", "LSSSEG2",
    "LPROD", "SEGMENT"
]

result9 = verify_schema(
    f"{REF_PATH}/lob.csv",
    "LOB_SCHEMA",
    schema_cols,
    "LOB - Line of Business"
)

## 10. CPRODUIT_SCHEMA - Product Reference

In [ ]:
# From schemas.py lines 352-357
schema_cols = [
    "cprod", "Type_Produit_2", "segment", "Segment_3"
]

result10 = verify_schema(
    f"{REF_PATH}/cproduit.csv",
    "CPRODUIT_SCHEMA",
    schema_cols,
    "CPRODUIT - Product reference"
)

## 11. GARANTCU_SCHEMA

In [ ]:
# From schemas.py lines 364-368
schema_cols = ["POLICE", "GARANTIE", "BRANCHE"]

result11 = verify_schema(
    f"{REF_PATH}/garantcu.csv",
    "GARANTCU_SCHEMA",
    schema_cols,
    "GARANTCU - Guarantee reference"
)

## 12. CATMIN_SCHEMA

In [ ]:
# From schemas.py lines 375-379
schema_cols = ["PRODUIT", "GARANTIE", "CATMIN5"]

result12 = verify_schema(
    f"{REF_PATH}/import_catmin.csv",
    "CATMIN_SCHEMA",
    schema_cols,
    "CATMIN - Category minimum"
)

## 13. POLIC_CU_SCHEMA - AZEC Policy Master

In [ ]:
# From schemas.py lines 385-412
schema_cols = [
    "CODECOAS", "CPCUA", "DATAFN", "DATEXPIR", "DATFIN", "DATRESIL", "DATTERME",
    "DUREE", "ECHEANJJ", "ECHEANMM", "EFFETPOL", "ETATPOL", "FINPOL",
    "GESTSIT", "INDREGUL", "INTERMED", "MOTIFRES", "NOMCLI", "ORIGRES",
    "PARTBRUT", "POINGEST", "POLICE", "PRIME", "PRODUIT", "RMPLCANT", "TYPCONTR"
]

result13 = verify_schema(
    f"{REF_PATH}/polic_cu.csv",
    "POLIC_CU_SCHEMA",
    schema_cols,
    "POLIC_CU - AZEC policy master"
)

## 14. CAPITXCU_SCHEMA - AZEC Capital Data

In [ ]:
# From schemas.py lines 418-425
schema_cols = [
    "POLICE", "PRODUIT", "SMP_SRE", "BRCH_REA", "CAPX_100", "CAPX_CUA"
]

result14 = verify_schema(
    f"{REF_PATH}/capitxcu.csv",
    "CAPITXCU_SCHEMA",
    schema_cols,
    "CAPITXCU - AZEC capital data"
)

## 15. AZEC_FORMULE_SCHEMA (RCENTCU + RISTECCU)

In [ ]:
# From schemas.py lines 433-440
schema_cols = [
    "POLICE", "PRODUIT", "COD_NAF", "FORMULE", "FORMULE2", "FORMULE3", "FORMULE4"
]

result15a = verify_schema(
    f"{REF_PATH}/rcentcu.csv",
    "AZEC_FORMULE_SCHEMA",
    schema_cols,
    "RCENTCU - RC Enterprise"
)

result15b = verify_schema(
    f"{REF_PATH}/risteccu.csv",
    "AZEC_FORMULE_SCHEMA",
    schema_cols,
    "RISTECCU - Professional risk"
)

## 16. MULPROCU_SCHEMA

In [ ]:
# From schemas.py lines 446-449
schema_cols = ["POLICE", "CHIFFAFF"]

result16 = verify_schema(
    f"{REF_PATH}/mulprocu.csv",
    "MULPROCU_SCHEMA",
    schema_cols,
    "MULPROCU - Multi-risk professional"
)

## 17. MPACU_SCHEMA

In [ ]:
# From schemas.py lines 455-458
schema_cols = ["POLICE", "COD_NAF"]

result17 = verify_schema(
    f"{REF_PATH}/mpacu.csv",
    "MPACU_SCHEMA",
    schema_cols,
    "MPACU - MPA policy data"
)

## 18. TABLE_PT_GEST_SCHEMA - Management Points (Versioned)

In [ ]:
# From schemas.py lines 537-540
# ⚠️ This is the one with upper_mid case issue!
schema_cols = ["PTGST", "UPPER_MID"]

result18 = verify_schema(
    f"{REF_PATH}/ptgst_*.csv",
    "TABLE_PT_GEST_SCHEMA",
    schema_cols,
    "TABLE_PT_GEST - Management points (versioned)"
)

## 19. PTGST_STATIC_SCHEMA - Management Points (Static)

In [ ]:
# From schemas.py lines 544-548
schema_cols = ["PTGST", "REGION", "P_NUM"]

result19 = verify_schema(
    f"{REF_PATH}/ptgst.csv",
    "PTGST_STATIC_SCHEMA",
    schema_cols,
    "PTGST_STATIC - Management points (static)"
)

## 20. PRDCAP_SCHEMA - Product Capitals Reference

In [ ]:
# From schemas.py lines 557-560
schema_cols = ["CDPROD", "LBTPROD"]

result20 = verify_schema(
    f"{REF_PATH}/prdcap.csv",
    "PRDCAP_SCHEMA",
    schema_cols,
    "PRDCAP - Product capitals reference"
)

## 21. SEGMENTPRDT_SCHEMA - Consolidated Product Segmentation

In [ ]:
# From schemas.py lines 567-573
schema_cols = ["CPROD", "CDPOLE", "CMARCH", "CSEG", "CSSSEG"]

result21 = verify_schema(
    f"{REF_PATH}/segmentprdt_*.csv",
    "SEGMENTPRDT_SCHEMA",
    schema_cols,
    "SEGMENTPRDT - Consolidated segmentation"
)

---
# Summary Report
---

In [ ]:
# Collect all results
all_results = [
    r for r in [
        result1, result2a, result2b, result3, result4a, result4b, result5,
        result6, result7, result8a, result8b, result9, result10, result11, result12,
        result13, result14, result15a, result15b, result16, result17, result18, result19,
        result20, result21
    ] if 'error' not in r
]

print("\n" + "="*100)
print("FINAL SUMMARY - CASE MISMATCH CORRECTIONS NEEDED")
print("="*100)

files_with_issues = [r for r in all_results if r['case_mismatch'] > 0 or r['missing'] > 0]

if files_with_issues:
    print(f"\n❌ {len(files_with_issues)} files need schema corrections:\n")
    for r in files_with_issues:
        print(f"\n{r['schema']} ({r['file']}):")
        if r['case_mismatch'] > 0:
            print(f"  ⚠️ {r['case_mismatch']} case mismatches to fix:")
            for schema_col, actual_col in r['mismatches']:
                print(f"     Change '{schema_col}' → '{actual_col}'")
        if r['missing'] > 0:
            print(f"  ✗ {r['missing']} columns missing from CSV")
else:
    print("\n✅ All schemas match perfectly!")

print("\n" + "="*100)